In [ ]:
import numpy as np


def fitness_function(X, bench_function):
  '''
  X : matrix of size (pop_size, num_dim) -> our population

  returns a matrix of size (pop_size,1) where each member in row is the fitness
  score of that individual form population
  '''
  return bench_function(X).reshape(-1,1)


def initialize_population(pop_size, dim_num, lw_bound, up_bound):
    '''
    this function will initilize population :
    pop_size : int -> how many inidviduals are in our population
    num_dim : int -> number of dimensions each individual has
    u_bound & l_bound : int -> for the boudary of our individuals

    returns a matrix of size <(pop_size*num_dim) where each row is an individual
    '''
    return np.random.rand(pop_size, dim_num) * (up_bound - lw_bound) + lw_bound


def evaluate_population(X, bench_function, minimize=True):
    '''
    X : matrix of population
    minimize : bool -> if its true our task is for finding individual with min(fitness_score)
    else reverse

    returns fitness_scores and most suited individual and its score
    '''
    fitness_values = fitness_function(X, bench_function)

    if minimize:
        best_index = np.argmin(fitness_values)
    else:
        best_index = np.argmax(fitness_values)

    best_fitness = fitness_values[best_index]
    best_solution = X[best_index, :]

    return fitness_values, best_fitness, best_solution



def mutation(population, mutation_fac):
    """Generate mutation vectors for entire population"""

    size = population.shape

    indices = np.random.choice(population.shape[0], size=size[0], replace=True)
    x1 = population[indices]
    indices = np.random.choice(population.shape[0], size=size[0], replace=True)
    x2 = population[indices]
    indices = np.random.choice(population.shape[0], size=size[0], replace=True)
    x3 = population[indices]

    return x1 + mutation_fac * (x2 - x3)


def crossover(X, mutant, CR):
    '''
    Performs crossover between the current population X and the mutant population,
    using a crossover rate CR. Additionally, for each individual, one randomly selected
    dimension is guaranteed to come from the mutant.

    Parameters:
      X      : ndarray of shape (pop_size, num_dim) - current population.
      mutant : ndarray of shape (pop_size, num_dim) - mutant population.
      CR     : float or ndarray broadcastable to X.shape - crossover rate.

    Returns:
      trial_vector : ndarray of shape (pop_size, num_dim) - the trial population after crossover.
    '''
    random_matrix = np.random.rand(*X.shape)
    trial_vector = np.where(random_matrix < CR, mutant, X)

    d_rand = np.random.randint(0, X.shape[1], size=(X.shape[0],))
    rows = np.arange(X.shape[0])
    trial_vector[rows, d_rand] = mutant[rows, d_rand]

    return trial_vector


def selection(X, trial, bench_function, minimize=True):
    '''
    X : our first population
    trial : the trial of same population after mutation and crossover
    indices are the ones when we split pop to 3 we will use it to update MF and MCR

    returns next gen population and items needed for updating archives
    '''
    fitness_X = fitness_function(X, bench_function)
    fitness_trial = fitness_function(trial, bench_function)

    if minimize:
        mask = (fitness_X > fitness_trial).flatten()
    else:
        mask = (fitness_trial > fitness_X).flatten()

    new_population = np.where(mask.reshape(-1, 1), trial, X)
    fitness = np.where(mask.reshape(-1,1), fitness_trial, fitness_X)

    return new_population, fitness


In [ ]:

def differential_evolution(num_dim, l_bound, u_bound, pop_size, bench_function, F = 0.5, CR = 0.8, verbose=False):

    # Initialize
    population = initialize_population(pop_size, num_dim, l_bound, u_bound)
    fitness_scores, fg_best, g_best = evaluate_population(population, bench_function, minimize=True)

    best_solution = g_best.copy()
    best_fitness = fg_best
    fes_max = 1000
    fes = 0
    while fes_max >=fes :

        # Mutation
        mutation_vectors = mutation(population, F)

        # Crossover
        trial_vectors = crossover(population, mutation_vectors, CR)

        # Selection
        population, fitness_scores = selection(population, trial_vectors, bench_function)

        if np.min(fitness_scores) < best_fitness:
                best_solution = population[np.argmin(fitness_scores)]
                best_fitness = np.min(fitness_scores)
        fes +=1

        if verbose:
                print(f"Generation {gnbg.n_fe}, Best Fitness: {best_fitness}, Best solution : {best_solution}")

    return best_solution, best_fitness